In [1]:
# def categorize_dnn_output(df, score_name, channel, region, year):
def categorize_dnn_output(df, score_name, region, year="2018"):
    """
    we only need VBF sample in h-peak region for this. No ggh, no sidebands
    """
    channel="vbf"
    # Run 2 (VBF yields)
    target_yields = {
        "2016": [
            0.35455259,
            0.50239086,
            0.51152889,
            0.52135985,
            0.5282209,
            0.54285134,
            0.54923751,
            0.56504687,
            0.57204477,
            0.58273066,
            0.5862248,
            0.59568793,
            0.60871905,
        ],
        "2017": [
            0.44194544,
            0.55885064,
            0.57796123,
            0.58007343,
            0.58141001,
            0.58144682,
            0.57858609,
            0.59887533,
            0.59511901,
            0.59644831,
            0.59825915,
            0.59283309,
            0.57329743,
        ],
        "2018": [
            0.22036263,
            1.31808978,
            1.25396849,
            1.183724,
            1.12620194,
            1.06041376,
            0.99941623,
            0.93224412,
            0.87074753,
            0.80599462,
            0.73469265,
            0.668018,
            0.60991945,
        ],
    }

    bins = [df[score_name].max()]

    # slicer = (
    #     (df.channel_nominal == channel) & (df.region == region) & (df.year == int(year))
    # )
    slicer = (
        (df.region == region)
    )
    df_sorted = (
        df.loc[slicer, :]
        .sort_values(by=score_name, ascending=False)
        .reset_index(drop=True)
    )
    df_sorted["wgt_cumsum"] = df_sorted.wgt_nominal.cumsum()

    tot_yield = 0
    last_yield = 0
    for yi in reversed(target_yields[year]):
        tot_yield += yi
        print(f"yi: {yi}")
        for i in range(df_sorted.shape[0] - 1):
            value = df_sorted.loc[i, "wgt_cumsum"]
            value1 = df_sorted.loc[i + 1, "wgt_cumsum"]
            if (value < tot_yield) & (value1 > tot_yield):
                if abs(last_yield - tot_yield) < 1e-06:
                    continue
                if abs(tot_yield - sum(target_yields[year])) < 1e-06:
                    continue
                bins.append(df_sorted.loc[i, score_name])
                last_yield = tot_yield
    bins.append(0.0)
    bins = sorted(bins)
    print(bins)

In [2]:
import awkward as ak
import dask_awkward as dak
import pandas as pd

In [3]:
global_path = "/depot/cms/users/yun79/hmm/copperheadV1clean/"
label = "DmitryMaster_JECoff_GeofitFixed_Nov01"

label = "rereco_yun_Nov04"
year="2018"
# full_load_path = f"{global_path}/{label}/stage2_unbinned/vbf_2018/vbf_powheg_dipole*.parquet"
full_load_path = f"{global_path}/{label}/stage2_unbinned/vbf_{year}/vbf_powheg_dipole*.parquet"
events = dak.from_parquet(full_load_path)

In [9]:
label

'rereco_yun_Nov04'

In [4]:
# ! ls /depot/cms/users/yun79/hmm/copperheadV1clean/DmitryMaster_JECoff_GeofitFixed_Nov01/stage2_unbinned/vbf_2018

In [5]:
events.fields

['dimuon_mass',
 'event',
 'wgt_nominal',
 'mu1_pt',
 'score_pytorch_jun27_nominal',
 'region',
 'year',
 'entry']

In [6]:
df  = ak.to_dataframe(ak.zip({
    "score_pytorch_jun27_nominal":events.score_pytorch_jun27_nominal,
    "region" :events.region,
    "wgt_nominal": events.wgt_nominal,
})).compute()

In [7]:
# df

In [8]:

categorize_dnn_output(df, "score_pytorch_jun27_nominal", "h-peak", year=year)

yi: 0.60991945
yi: 0.668018
yi: 0.73469265
yi: 0.80599462
yi: 0.87074753
yi: 0.93224412
yi: 0.99941623
yi: 1.06041376
yi: 1.12620194
yi: 1.183724
yi: 1.25396849
yi: 1.31808978
yi: 0.22036263
[0.0, 0.07355153560638428, 0.48736143112182617, 0.7758049964904785, 0.996168315410614, 1.1832327842712402, 1.3480875492095947, 1.4972871541976929, 1.6352977752685547, 1.762912392616272, 1.8872747421264648, 2.005509614944458, 2.1303319931030273, 2.474616050720215]
